In [1]:
import numpy as np
import math
import xarray as xr
import os
import tensorflow as tf
import xbatcher as xb
import xbatcher.loaders.keras
import copy

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from cnn import CNN, Scenario, ReplicationPadding2D, MaskedMSELoss
import preprocess_data

2026-02-03 10:41:42.957987: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-03 10:41:43.002313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-03 10:42:26.698090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Some functions

In [2]:
def _add_dimension_and_slice(ds, x_slice, y_slice, region):
    '''
        Adds a dimension to allow all regional datasets to 
            be combined.
        Slice the data based on input x and y.
    '''
    ds = ds.expand_dims({"r": 1})
    for coord in ds.coords:
        if coord != "t": #* omit t dimension
            ds[coord] = ds[coord].expand_dims({"r": 1})

    ds_tmp = ds.isel(x_c=x_slice, y_c=y_slice)

    return ds_tmp

In [3]:
def _slice_data(ds, x_slice, y_slice):
    '''
        Slice the data based on input x and y.
    '''

    ds_tmp = ds.isel(x_c=x_slice, y_c=y_slice)

    return ds_tmp

In [4]:
def open_and_process_data(scenario, directory, filenames, domain):
    '''
        Opens and preprocesses the data by feeding in directory and filenames
            into the preprocessing step.
            n.b. this only provides local normalization related to each sub domain.
                use open_and_combine_data then feed in datasets for global 
                normalization.
    '''
    ds = {}

    sc = copy.deepcopy(scenario) # make a copy
    print(sc)

    for region in domain:

        directory_region = directory.format(domain=region)
        fnames_region = [f.format(domain=region) for f in filenames]

        processor = preprocess_data.data_preparation(scenario, 
                                                     directory=directory_region, 
                                                     filenames=fnames_region,
                                                     parallel=False,
                                                     )
        ds[region] = processor()
        # print(ds[region])

        scenario = copy.deepcopy(sc)
        print(scenario)

    dom_slice_dict = {
        'dDP': (slice(2, 37), slice(30, 65)),
        'uDP': (slice(2, 37), slice(10, 45)),
        'SP': (slice(2, 37), slice(15, 50)),
        'IO': (slice(2, 37), slice(15, 50)),
        'SO_JET': (slice(3, 43), slice(7, 47)),
        }
    
    for region in domain:
        ds_region = _slice_data(ds[region], 
                                 dom_slice_dict[region][0],
                                 dom_slice_dict[region][1],
                                )

        ds[region] = ds_region

    dataset_list = [ds[region] for region in domain]

    ds_combined = xr.concat(dataset_list, dim='r')

    return ds_combined

Set up the scenario

In [5]:
sc = Scenario(['mke', 'vor', 'sa', 'eke_shift'], ['eke'],
              [256, 128, 64, 32, 32, 1], 
              [(5,5), (3,3), (3,3), (3,3), (3,3), (1,1)], 
              [(2,2), (1,1), (1,1), (1,1), (1,1), (0,0)],
              name = 'testing')

Open and process data using local normalization

In [6]:
directory = "/gws/nopw/j04/ai4pex/twilder/NEMO_data/DINO/EXP16/features/{domain}/coarsened_data/"

fns = ["MINT_1d_0061-0072_sa_c_{domain}.nc",
       "MINT_1d_0061-0072_vor_cg_{domain}_mod.nc",
       "MINT_1d_0061-0072_eke_c_{domain}.nc",
       "MINT_1d_0061-0072_eke_c_{domain}_shifted.nc",
       "MINT_1d_0061-0072_mke_c_{domain}.nc",
       "mesh_mask_exp4_{domain}_xnemo.nc"]

domain = ['SO_JET']

ds = open_and_process_data(sc, directory, fns, domain)

Scenario(input_var=['mke', 'vor', 'sa', 'eke_shift'], target=['eke'], filters=[256, 128, 64, 32, 32, 1], kernels=[(5, 5), (3, 3), (3, 3), (3, 3), (3, 3), (1, 1)], padding=[(2, 2), (1, 1), (1, 1), (1, 1), (1, 1), (0, 0)], name='testing')
Scenario(input_var=['mke', 'vor', 'sa', 'eke_shift'], target=['eke'], filters=[256, 128, 64, 32, 32, 1], kernels=[(5, 5), (3, 3), (3, 3), (3, 3), (3, 3), (1, 1)], padding=[(2, 2), (1, 1), (1, 1), (1, 1), (1, 1), (0, 0)], name='testing')


In [7]:
ds

<xarray.Dataset> Size: 442MB
Dimensions:        (r: 1, t: 4320, y_c: 40, x_c: 40)
Coordinates:
    x_c            (r, x_c) int64 320B 3 4 5 6 7 8 9 10 ... 36 37 38 39 40 41 42
    y_c            (r, y_c) int64 320B 7 8 9 10 11 12 13 ... 41 42 43 44 45 46
  * t              (t) object 35kB 0061-01-02 00:00:00 ... 0073-01-01 00:00:00
    gphit          (r, y_c, x_c) float64 13kB -46.22 -46.22 ... -39.07 -39.07
    glamt          (r, y_c, x_c) float64 13kB 3.375 3.625 3.875 ... 12.88 13.12
    gdept_1d       (r, y_c, x_c) float64 13kB 5.034 5.034 5.034 ... 5.034 5.034
    gdept_0        (r, y_c, x_c) float64 13kB 5.034 5.034 5.034 ... 5.034 5.034
    z_c            (r) int64 8B 0
Dimensions without coordinates: r
Data variables:
    sa             (r, t, y_c, x_c) float64 55MB -1.749 -1.715 ... 0.2151 0.1187
    vor            (r, y_c, x_c, t) float64 55MB -0.08827 0.09387 ... -0.4266
    eke            (r, t, y_c, x_c) float64 55MB 0.005281 0.006964 ... 0.009702
    eke_shift      (r, t, y_c, x_c) float64 55MB 0.00877 0.009514 ... nan nan
    mke            (r, t, y_c, x_c) float64 55MB 0.03049 0.02022 ... 0.01001
    mke_log        (r, t, y_c, x_c) float64 55MB -0.01738 -0.2697 ... -0.7012
    eke_shift_log  (r, t, y_c, x_c) float64 55MB -1.133 -1.074 ... 2.3 2.3
    eke_log        (r, t, y_c, x_c) float64 55MB -5.244 -4.967 ... -4.481 -4.635

In [10]:
directory = "/gws/nopw/j04/ai4pex/twilder/NEMO_data/DINO/EXP16/features/SO_JET/coarsened_data/"
ds.to_netcdf(directory + 'preprocessed_SO_JET_data.nc')

Subset the data for prediction

In [8]:
# ----------------------------
# 2. Define split sizes
# ----------------------------
n_test = 359     # last 30 days
n_val  = 360     # 60 days before test
train_stride = 5  # every 4th day

nt = ds.sizes["t"]

# ----------------------------
# 3. Create time indices
# ----------------------------
test_idx = np.arange(nt - n_test, nt)
val_idx  = np.arange(nt - n_test - n_val, nt - n_test)
train_idx_full = np.arange(0, nt - n_test - n_val)

# ----------------------------
# 4. Subsample training every 4th day
# ----------------------------
train_idx = train_idx_full[::train_stride]

# ----------------------------
# 5. Create split datasets
# ----------------------------
ds_train = ds.isel(t=train_idx)
ds_val   = ds.isel(t=val_idx)
ds_test  = ds.isel(t=test_idx)

print("Train:", ds_train.sizes["t"])
print("Val:  ", ds_val.sizes["t"])
print("Test: ", ds_test.sizes["t"])

Train: 721
Val:   360
Test:  359


In [21]:
ds_train

<xarray.Dataset> Size: 55MB
Dimensions:   (r: 1, t: 720, y_c: 40, x_c: 40)
Coordinates:
    y_c       (r, y_c) int64 320B 7 8 9 10 11 12 13 14 ... 40 41 42 43 44 45 46
    x_c       (r, x_c) int64 320B 3 4 5 6 7 8 9 10 ... 35 36 37 38 39 40 41 42
  * t         (t) object 6kB 0061-01-02 00:00:00 ... 0070-12-27 00:00:00
    gphit     (r, y_c, x_c) float64 13kB -46.22 -46.22 -46.22 ... -39.07 -39.07
    glamt     (r, y_c, x_c) float64 13kB 3.375 3.625 3.875 ... 12.62 12.88 13.12
    gdept_1d  (r, y_c, x_c) float64 13kB 5.034 5.034 5.034 ... 5.034 5.034 5.034
    gdept_0   (r, y_c, x_c) float64 13kB 5.034 5.034 5.034 ... 5.034 5.034 5.034
    z_c       (r) int64 8B 0
Dimensions without coordinates: r
Data variables:
    sa        (r, t, y_c, x_c) float64 9MB -1.749 -1.715 -1.7 ... 0.7623 0.7845
    vor       (r, y_c, x_c, t) float64 9MB -0.08827 -0.01104 ... -0.08475 0.3774
    eke       (r, t, y_c, x_c) float64 9MB 0.005281 0.006964 ... 0.02981 0.03925
    mke       (r, t, y_c, x_c) float64 9MB 0.03049 0.02022 ... 0.001305 0.001425
    mke_log   (r, t, y_c, x_c) float64 9MB -0.01738 -0.2697 ... -1.952 -1.898
    eke_log   (r, t, y_c, x_c) float64 9MB -5.244 -4.967 ... -3.513 -3.238

Loading the trained model

In [9]:
# load in keras model
filename = 'training/cnn_20260203-092450.keras'
model = keras.saving.load_model(
    filename,
    compile=True,
)
model.summary()

2026-02-03 10:46:21.091001: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 40, 40, 128)    │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 40, 40, 64)     │       205,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 40, 40, 32)     │        18,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 40, 40, 16)     │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 40, 40, 8)      │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 40, 40, 1)      │            72 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 969,938 (3.70 MB)

 Trainable params: 242,360 (946.72 KB)

 Non-trainable params: 496 (1.94 KB)

 Optimizer params: 727,082 (2.77 MB)

In [10]:
ds_prediction = ds_test.isel(r=0)

Now making a prediction

In [11]:
# retrieve input and taget variables
batch_input  = [ds_prediction[x] for x in sc.input_var]
batch_target  = [ds_prediction[x] for x in sc.target]
# adds an additional dimension for tf readability
batch_input  = xr.merge(batch_input).to_array('var').transpose(...,'var') # channels
batch_target  = xr.merge(batch_target).to_array('var').transpose(...,'var')

target = model.predict(batch_input.to_numpy())

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step


In [12]:
pred_ds = xr.Dataset(
        data_vars={
            "fine_ke_pred": (['time_counter', 'y', 'x', 'var'], np.exp(target)),  # Adjusted to match the model output
            "fine_ke_true": (['time_counter', 'y', 'x', 'var'], np.exp(batch_target.to_numpy())),
        },
        coords={
            "time_counter": (["time_counter"], ds_prediction.t.values, 
                             ds_prediction.t.attrs),
            "gphit": (["y", "x"], ds_prediction.gphit.values, 
                      {"standard_name": "Latitude", "units": "degrees_north"}),
            "glamt": (["y", "x"], ds_prediction.glamt.values, 
                      {"standard_name": "Longitude","units": "degrees_east"}),
            'var': sc.target,
        },
        attrs={
            'Title': 'Fine kinetic energy - predicted and truth (region IO)',
            'Description': 'Predicted fine kinetic energy from coarse-grained data using CNN',
            'Units': 'm^2/s^2',
            'Source': f'{filename}',
        }
    )

In [13]:

pred_ds.to_netcdf('predictions/eddy_energy_20260203-092450_SO_JET.nc')